In [79]:
#!pip install keras --upgrade
''''''''''

# File Paths

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


!mkdir -p drive
!google-drive-ocamlfuse drive
'''''''''



"'\n\n# File Paths\n\n!apt-get install -y -qq software-properties-common python-software-properties module-init-tools\n!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null\n!apt-get update -qq 2>&1 > /dev/null\n!apt-get -y install -qq google-drive-ocamlfuse fuse\n\nfrom google.colab import auth\nauth.authenticate_user()\nfrom oauth2client.client import GoogleCredentials\ncreds = GoogleCredentials.get_application_default()\nimport getpass\n\n!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL\nvcode = getpass.getpass()\n!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}\n\n\n!mkdir -p drive\n!google-drive-ocamlfuse drive\n"

In [80]:
import os

traindir = 'train'
testdir = 'test'

#know your Data

train_b_dir = os.path.join(traindir, 'Blight')  # directory with our training cat pictures
train_h_dir = os.path.join(traindir, 'Healthy')  # directory with our training dog pictures

validation_b_dir = os.path.join(testdir, 'Blight')  # directory with our validation cat pictures
validation_h_dir = os.path.join(testdir, 'Healthy')  # directory with our validation dog pictures

num_b_tr = len(os.listdir(train_b_dir))
num_h_tr = len(os.listdir(train_h_dir))

num_b_val = len(os.listdir(validation_b_dir))
num_h_val = len(os.listdir(validation_h_dir))

total_train = num_b_tr + num_h_tr
total_val = num_b_val + num_h_val

print(total_train)
print(total_val)

200
80


# Potato_Classification

In [81]:
import pandas as pd
import numpy as np

#NN
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping

#CNN

from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.preprocessing.image import ImageDataGenerator


#plot
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()


from keras import layers
from keras import models
from keras import optimizers

*Preparing Model*

In [104]:
# Building the Model 

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (64, 64, 3),  kernel_initializer=tf.keras.initializers.Zeros()))
model.add(tf.keras.layers.MaxPooling2D(2, 2))

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2))

model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))


#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [98]:


# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),

#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),

#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),

#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),

#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(2)
# ])

In [105]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 6272)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)               802944    
__________

In [106]:
# prepared by Expert

rms = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9)
model.compile(optimizer = rms, loss = 'binary_crossentropy', metrics = ['acc'])



#prepared by me:

#adam = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])


*Data Augmentation*

* Shearing of images
* Random zoom
* Horizontal flips

In [107]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

#Training Set
train_set = train_datagen.flow_from_directory(traindir,
                                             target_size=(64,64),
                                             batch_size=32,
                                             class_mode='binary')

#Test Set /no output available
test_set = test_datagen.flow_from_directory(testdir,
                                            target_size=(64,64),
                                            batch_size=32,
                                            shuffle=False)


Found 200 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [108]:
# showing the output for the above code

for data_batch, labels_batch in train_set:
  print('Data Batch Shape', data_batch.shape)
  print('Labels Batch Shape', labels_batch.shape)
  break

Data Batch Shape (32, 64, 64, 3)
Labels Batch Shape (32,)


In [109]:
# Time for Training:

BATCH_SIZE = 10
epochs=100
history = model.fit_generator(
    train_set,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=test_set,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)



#history = model.fit_generator(train_set,
#                         steps_per_epoch=10,
#                         epochs = 30,
#                         validation_data = test_set,
#                         validation_steps = 5, 
                        #callbacks=[tensorboard]
#                        );

# steps_per_epoch = 2000/20 = 100, validation_steps = 1000/20 = 50



#history = model.fit_generator(train_set, steps_per_epoch = 100, epochs = 30, validation_data = test_set, validation_steps = 50)


Epoch 1/100
6/7 [========================>.....] - ETA: 0s - loss: 0.6930 - acc: 0.5060

ValueError: A target array with shape (32, 2) was passed for an output of shape (None, 1) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.

# Accuracy Report

In [ ]:
acc = history.history['acc']
test_acc = history.history['val_acc']
loss = history.history['loss']
test_loss = history.history['val_loss']

# Plotting Training and Testing Loss

epochs = range(1, 31)
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, test_loss, 'b', label='Testing Loss')
plt.title('Training and validation Loss')
plt.legend()
plt.show()

In [ ]:
# Plotting Training and Testing Accuracy

plt.plot(epochs, acc, 'ko', label='Training acc')
plt.plot(epochs, test_acc, 'k', label='Testing acc')
plt.title('Training and Testing accuracy')
plt.legend()
plt.show()

In [ ]:
# Testing

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=20, class_mode='binary')
test_loss, test_acc = model1.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)